# 1  Sample metadata

Explore the sample metadata to familiarize yourself with the samples in detail. QIIME 2’s metadata plugin provides a Visualizer called tabulate that generates a convenient view of a sample metadata file.

See the Qiime documentation for details on the metadata file format. You can validate the metadata table in Google Sheets using the Keemei plugin.

Additional Information: If you plan to share the Qiime result files with others, don't include confidential information (e.g. Personally Identifying Information, PII), in the metadata. Instead, encode it with variables that only authorized individuals have access to. For example, subject names should be replaced with anonymized subject identifiers before use with QIIME 2.

In [10]:
# Install openpyxl to read Excel files (using --user since environment is read-only)
%pip install --user -q openpyxl

Note: you may need to restart the kernel to use updated packages.


In [11]:
# Add user site-packages to path so we can import openpyxl
import sys
import site
user_site = site.getusersitepackages()
if user_site not in sys.path:
    sys.path.insert(0, user_site)
    
# Now try importing openpyxl
import openpyxl
print(f"✓ openpyxl {openpyxl.__version__} loaded successfully from {user_site}")

✓ openpyxl 3.1.5 loaded successfully from /home/fhwn.ac.at/211567/.local/lib/python3.9/site-packages


In [12]:
import pandas as pd
import os

# Load the metadata Excel file
fn = '/home/fhwn.ac.at/211567/fallstudie/EOTRH-MetadatenProben-WS2025.xlsx'
df = pd.read_excel(fn, header=1, engine='openpyxl')

# Display basic information about the metadata
print(f"Metadata shape: {df.shape[0]} samples, {df.shape[1]} columns")
print(f"\nColumn names:\n{list(df.columns)}")
print(f"\nFirst few rows:")
df.head()

Metadata shape: 23 samples, 11 columns

Column names:
['Seq Pos', 'Abbr', 'Horse', 'Type', 'Tooth #', 'Tooth location', 'Replicate', 'Gender', 'Age', 'disease state', 'DIN']

First few rows:


,Seq Pos,Abbr,Horse,Type,Tooth #,Tooth location,Replicate,Gender,Age,disease state,DIN
0,1,KP1,Kommi,Plaque,1,upper jaw left,1,W,10,healthy,7.6
1,2,KP2,Kommi,Plaque,2,upper jaw rigth,2,W,10,healthy,6.7
2,3,KP3,Kommi,Plaque,3,bottom jaw right,3,W,10,healthy,6.8
3,4,KP4,Kommi,Plaque,1,up right,4,W,10,healthy,7.1
4,5,KP5,Kommi,Plaque,2,up left,5,W,10,healthy,7.1


In [13]:
# Format metadata for QIIME 2
# First, let's check what we're working with
print("Original columns:", list(df.columns))
print("Shape:", df.shape)

# 1. Copy and rename 'Abbr' column to '#SampleID' (required by QIIME 2)
df_qiime = df.copy()

# Check if 'Abbr' exists
if 'Abbr' in df_qiime.columns:
    df_qiime = df_qiime.rename(columns={'Abbr': '#SampleID'})
    print("✓ Renamed 'Abbr' to '#SampleID'")
else:
    print("⚠ Warning: 'Abbr' column not found!")
    print("Available columns:", list(df_qiime.columns))

# 2. Remove any rows where the sample ID is missing
df_qiime = df_qiime[df_qiime['#SampleID'].notna()]

# 3. Clean column names: remove spaces, special characters (except # in #SampleID)
new_columns = []
for col in df_qiime.columns:
    if col == '#SampleID':
        new_columns.append('#SampleID')
    else:
        new_columns.append(col.replace(' ', '_').replace('#', '').replace('.', ''))
df_qiime.columns = new_columns

# 4. Remove the 'Seq_Pos' column if present (it's just row numbers)
if 'Seq_Pos' in df_qiime.columns:
    df_qiime = df_qiime.drop('Seq_Pos', axis=1)
    print("✓ Removed 'Seq_Pos' column")

# 5. Save as TSV file (QIIME 2 metadata format)
output_file = 'sample-metadata.tsv'
df_qiime.to_csv(output_file, sep='\t', index=False)

print(f"\n✓ Saved QIIME 2 metadata to: {output_file}")
print(f"  Samples: {len(df_qiime)}")
print(f"  Columns: {list(df_qiime.columns)}")
print(f"\nFirst few rows:")
df_qiime.head()

Original columns: ['Seq Pos', 'Abbr', 'Horse', 'Type', 'Tooth #', 'Tooth location', 'Replicate', 'Gender', 'Age', 'disease state', 'DIN']
Shape: (23, 11)
✓ Renamed 'Abbr' to '#SampleID'
✓ Removed 'Seq_Pos' column

✓ Saved QIIME 2 metadata to: sample-metadata.tsv
  Samples: 23
  Columns: ['#SampleID', 'Horse', 'Type', 'Tooth_', 'Tooth_location', 'Replicate', 'Gender', 'Age', 'disease_state', 'DIN']

First few rows:


,#SampleID,Horse,Type,Tooth_,Tooth_location,Replicate,Gender,Age,disease_state,DIN
0,KP1,Kommi,Plaque,1,upper jaw left,1,W,10,healthy,7.6
1,KP2,Kommi,Plaque,2,upper jaw rigth,2,W,10,healthy,6.7
2,KP3,Kommi,Plaque,3,bottom jaw right,3,W,10,healthy,6.8
3,KP4,Kommi,Plaque,1,up right,4,W,10,healthy,7.1
4,KP5,Kommi,Plaque,2,up left,5,W,10,healthy,7.1


In [14]:
# Display summary statistics about the metadata
print("=" * 60)
print("METADATA SUMMARY")
print("=" * 60)
print(f"\nTotal samples: {len(df_qiime)}")
print(f"\nSample breakdown:")
print(f"  - Horses: {df_qiime['Horse'].nunique()} unique ({', '.join(map(str, df_qiime['Horse'].unique()))})")
print(f"  - Sample types: {df_qiime['Type'].value_counts().to_dict()}")
print(f"  - Disease states: {df_qiime['disease_state'].value_counts().to_dict()}")
print(f"  - Gender distribution: {df_qiime['Gender'].value_counts().to_dict()}")

# Handle Age - filter out non-numeric values
numeric_ages = pd.to_numeric(df_qiime['Age'], errors='coerce').dropna()
if len(numeric_ages) > 0:
    print(f"  - Age range: {numeric_ages.min():.0f} - {numeric_ages.max():.0f} years (n={len(numeric_ages)})")

# Handle DIN - filter out non-numeric values  
numeric_din = pd.to_numeric(df_qiime['DIN'], errors='coerce').dropna()
if len(numeric_din) > 0:
    print(f"  - DIN range: {numeric_din.min():.1f} - {numeric_din.max():.1f} (n={len(numeric_din)})")

print(f"\nMetadata columns: {', '.join([c for c in df_qiime.columns if c != '#SampleID'])}")

METADATA SUMMARY

Total samples: 23

Sample breakdown:
  - Horses: 5 unique (Kommi, Threnna, Eydis, E. coli, H2O)
  - Sample types: {'Plaque': 14, 'Gum': 7, '-': 2}
  - Disease states: {'diseased': 9, 'healthy': 7, 'onset': 5, '-': 2}
  - Gender distribution: {'S': 14, 'W': 7, '-': 2}
  - Age range: 10 - 24 years (n=21)
  - DIN range: 6.3 - 8.4 (n=22)

Metadata columns: Horse, Type, Tooth_, Tooth_location, Replicate, Gender, Age, disease_state, DIN


![Sample metadata screenshot](sample-metadata-screenshot.png)

# 2  Obtaining and importing data

Data is imported in Qiime as QIIME 2 "artifacts". They are simply zip files that store data together with some QIIME 2-specific metadata, e.g. type and source of data. The metadata are tracked by Qiime automatically, which makes working with artifacts very convenient. Artifacts have the file extension .qza.

All QIIME 2 artifacts are assigned one artifact class, which indicates the semantics of the data (its "semantic type") and the file format that is used to store it inside of the .qza file. When you see artifacts (or inputs or outputs to an action) described with terms that look like Phylogeny[Rooted] or Phylogeny[Unrooted], that is the Artifact Class.

The file extension .qzv (QIIME Zipped Visualization) are files that store QIIME 2 Visualizations. QIIME 2 Visualizations represent data that are generated by QIIME 2 and intended to be viewed by humans, such as an interactive visualization.

Details on data import are given in the How-To: Import data for use with QIIME 2. Notably, the guide recommends importing demultiplexed data (as in your case) using a FASTQ "manifest" file, which is a simple text file that maps sample identifiers to one or two absolute filepaths pointing at .fastq.gz (or .fastq) files, depending on whether you’re importing data from a single- or paried-end run.

In [ ]:
import zipfile

fn = '/home/fhwn.ac.at/211567/fallstudie/20241209-raw_data/HN00230849.zip'
with zipfile.ZipFile(fn) as zf:
    zf.extractall('emp-single-end-sequences')